# PROJET RI TESTS

In [1]:
import datasets
import utils

/home/neyroy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /home/neyroy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/neyroy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## DATA

In [9]:
#T elechargement et sauvegarde des datasets 

# from datasets import load_dataset 
# wikiqa = load_dataset("wiki_qa")
# wikiqa.save_to_disk("wikiqa")

In [2]:
#Load le dataset ( enregistré en format pyarrow ./wikiqa )
wikiqa_data = datasets.load_from_disk("wikiqa")

In [7]:
print(wikiqa_data)
print(wikiqa_data["train"])

DatasetDict({
    test: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 6165
    })
    validation: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 2733
    })
    train: Dataset({
        features: ['question_id', 'question', 'document_title', 'answer', 'label'],
        num_rows: 20360
    })
})
Dataset({
    features: ['question_id', 'question', 'document_title', 'answer', 'label'],
    num_rows: 20360
})


In [3]:
wikiqa_data_train = wikiqa_data["train"]    
questions = wikiqa_data_train["question"]   #questions
answers = wikiqa_data_train["answer"]       #answers
labels = wikiqa_data_train["label"]         #labels (pertinent non pertinent)

#Preprocessing ( see utils.preprocess )
questions = [utils.preprocess(txt) for txt in questions]
answers = [utils.preprocess(txt) for txt in answers]

In [4]:
#Length Number of terms in the sentence
def length(answer):
    return len(answer.split())

# ExactMatch Whether query is a substring
def check_exact_match(query, text):
    if query.lower() in text.lower():
        return 1  # Exact match found
    else:
        return 0  # No exact match found
    
#Overlap Fraction of query terms covered
def calculate_overlap_fraction(qrep, rep):
    overlap_fractions = []
    
    for qtext in qrep:
        qterms = set(qtext.lower().split())
        for text in rep:
            text = text.lower()
            tterms = set(text.split())
            overlap = len(qterms.intersection(tterms))
            fraction = overlap / len(qterms) if len(qterms) > 0 else 0
            overlap_fractions.append(fraction)
    
    return overlap_fractions

In [27]:
# test de la feature overlap
qrep = ["apples banana"]
rep = ["I like apples and bananas.", "I prefer oranges."]

# Calculate overlap fraction feature for the sample data using the adapted function
overlap_fraction_features = list(calculate_overlap_fraction(qrep, rep))

print(overlap_fraction_features)

[0.5, 0.0]


In [5]:
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk

nltk.download('wordnet')
nltk.download('stopwords')


# Overlap avec synonym : 

def get_synonyms(word):
    """
    get synonyms of a word
    """
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

def tokenize_text(text):
    """ 
    Tokenize the text 
    """
    #stop_words = set(stopwords.words('english'))
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    #tokens = [token for token in tokens if token not in stop_words]
    return tokens

def overlap_syn_fraction(qrep, rep):
    """
    This function calculates the overlap synonym fraction between a query and a set of answers by finding synonyms of words in the query
    and comparing them with the synonyms of words in each answer.

    qrep : questions - list of String
    rep : answers - list of String
"""
    q_synonyms = {word: utils.get_synonyms(word) for word in utils.tokenize_text(qrep)}
    q_synonyms_lens = {word: len(synonyms) for word, synonyms in q_synonyms.items()}
    for text in rep:
        text_tokens = utils.tokenize_text(text)
        t_synonyms = set()
        for token in text_tokens:
            t_synonyms.update(utils.get_synonyms(token))
        overlap = sum(1 for syn in q_synonyms if syn in t_synonyms)
        fraction = overlap / sum(q_synonyms_lens.values()) if sum(q_synonyms_lens.values()) > 0 else 0
        yield fraction

# Sample data
qrep = "sad"
rep = ["I am feeling deplorable today.", "She seems pitiful and sorry."]

# Calculate overlap synonym fraction feature for the sample data using the function
overlap_syn_fraction_features = list(overlap_syn_fraction(qrep, rep))

print(overlap_syn_fraction_features)

print(get_synonyms('sad'))

[nltk_data] Downloading package wordnet to /home/neyroy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/neyroy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[0.16666666666666666, 0.16666666666666666]
{'sorry', 'pitiful', 'lamentable', 'sad', 'deplorable', 'distressing'}


In [28]:
from rank_bm25 import BM25Okapi

def calculate_bm25_score(query, document):
    tokenized_query = query.lower().split()
    tokenized_document = document.lower().split()
    
    corpus = [tokenized_document]
    bm25 = BM25Okapi(corpus)
    
    #average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
    
    score = bm25.get_scores(tokenized_query)
    
    # # Sort the answers based on BM25 scores in descending order
    # sorted_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)
    # sorted_answers = [corpus[i] for i in sorted_indices]
    return score[0]  # Aif only one document in the corpus

# Sample data
query = "i like apples and bananas"
document = "I like apples and bananas."

# Calculate BM25 score for the sample query and document
bm25_score = calculate_bm25_score(query, document)

print("BM25 score:", bm25_score)

BM25 score: 0.0


In [8]:
import nltk
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.lm.preprocessing import pad_both_ends

nltk.download('gutenberg')
from nltk.corpus import gutenberg

# Train a unigram language model using Gutenberg corpus
text = gutenberg.raw('shakespeare-hamlet.txt')
tokens = text.split()
train_data = [list(ngrams(pad_both_ends(tokens, n=1), n=1)) for n in range(1, 2)]
lm = Laplace(1)  # Laplace smoothing with n=1
lm.fit(train_data)

def calculate_lm_score(text):
    test_data = list(ngrams(pad_both_ends(text.split(), n=1), 1))
    score = lm.perplexity(test_data)
    return score

# Sample data
text1 = "To be or not to be"
text2 = "To sleep, perchance to dream"

# Calculate LM score for the sample data
lm_score1 = calculate_lm_score(text1)
lm_score2 = calculate_lm_score(text2)

print("LM score for text1:", lm_score1)
print("LM score for text2:", lm_score2)

[nltk_data] Downloading package gutenberg to /home/neyroy/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


ValueError: Cannot fit without a vocabulary or text to create it from.